In [1]:
import censusgeocode as cg
import pandas as pd
import numpy as np
import json
import os
import time




/Users/Alex/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/Users/Alex/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [5]:
os.chdir('/Users/Alex/Library/Mobile Documents/com~apple~CloudDocs/School/Fall 19/CPSC 6300 Applied Data Science/Project/Analysis-of-Boston-Crime-Report-master/Boston_crime_library')


dat = pd.read_csv('DataSet/Boston_Crime_Data/Boston_Crime_Data.csv', engine = 'python')
    
#?pd.read_csv()

In [3]:
dat.shape

(296292, 12)

Remove NA values from Longitude and Latitude, round both up to three decimal places, and merge them into into a single DataFrame.


In [6]:
print(dat.shape)
dat = dat.dropna(subset = ['Long', 'Lat'])

dat.drop(dat.loc[dat['Lat']== -1].index, inplace=True)
dat.drop(dat.loc[dat['Long']== -1].index, inplace=True)

print(dat.shape)

dat.reset_index(inplace = True)




(319073, 17)
(298329, 17)


In [7]:


Lat = pd.to_numeric(dat.loc[:,'Lat'])
Long = pd.to_numeric(dat.loc[:,'Long'])


Lat = np.round(Lat, 3)
Long = np.round(Long, 3)

print(dat.loc[42,['Lat']], dat.loc[42,['Long']])


Lat    42.2849
Name: 42, dtype: object Long   -71.0714
Name: 42, dtype: object


In [10]:
coord_dat = pd.concat([Long, Lat], axis = 1)
before = coord_dat.shape
coord_dat = coord_dat.drop_duplicates()
after = coord_dat.shape
print(f'Before: {before}, After: {after}')
print(f'When coordinates are rounded down to 3 places, we only have {after[0]} unique coordinates...')
Long = coord_dat.iloc[:,[0]]
Lat = coord_dat.iloc[:,[1]]


Before: (7757, 2), After: (7757, 2)
When coordinates are rounded down to 3 places, we only have 7757 unique coordinates...


raise_wrapper allows the user to make repeated requests for tract info from the Census. Occaisionally, requests will return a ValueError  and terminate all further requests, despite the code being error free.
Wrapping requests in a function allows make repeated requests with the same coordinates for up to num_retries times.

In [11]:

def raise_wrapper(num_retries, index):
    for attempt_no in range(num_retries):
        try:
            #time.sleep(float(np.sqrt(np.random.randn(1)**2)))
            return cg.coordinates(float(Long.iloc[index]), float(Lat.iloc[index]), returntype = 'georgraphies')['Census Tracts'][0]['TRACT']

         
        except ValueError:
            print(f'ValueError @ item {i}!')
            time.sleep(float(np.sqrt(np.random.randn(1)**2) + 5))
            if attempt_no < (num_retries - 1):
                print(f'err no {attempt_no}')
            else:
                raise error
 



Requests are sent to the census, appended to a list, Census_Data, for as many unique requests there are (i.e. number of rows in Long and Lat). Note that this will take some time to complete. Once it is done, the results are stored as a JSON file, Crime_Location_Data.txt.

Additionally, a csv is created in the code block below that contains these Tract IDs, as well as the longitude and latitudes use to make the requests. This data will be used to add a Tract column to the Boston Dataset, and then merge with the ACS Survey data.

In [14]:
Census_Data = []
try:
    for i in range(0,len((Lat))): 
        geocode = raise_wrapper(num_retries = 15,index = i)
        if i % 100 == 0:
            print(f'Report {i} geocoded.')
#        print(f'Coordinates for row {i} geocoded.')
        Census_Data.append(geocode)

finally:
    with open('Crime_Location_Data.txt', 'w') as CLD:
        json.dump(Census_Data, CLD)    

Report 0 geocoded.


In [15]:

census_tracts = np.array(Census_Data)
census_tracts = census_tracts[...,np.newaxis]

Longitude = Long.T.squeeze().to_numpy()
Longitude = Longitude[...,np.newaxis]

Latitude = Lat.T.squeeze().to_numpy()
Latitude = Latitude[...,np.newaxis]

coord_map = np.hstack([census_tracts, Longitude, Latitude])

census_tracts = pd.Series(coord_map[:,0])
Longitude = pd.Series(coord_map[:,1])
Latitude = pd.Series(coord_map[:,2])

coord_map = pd.concat([census_tracts, Longitude, Latitude], axis = 1)
#print(coord_map.shape, type(coord_map))

coord_map.rename(columns = {0:'Census_Tract', 1:'Longitude', 2:'Latitude'}, inplace = True)
coord_map.head()

coord_map.to_csv('DataSet/Boston_Crime_Data/Tracts_Coords_For_Merge.csv')

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 3 and the array at index 1 has size 7757